In [1]:
import pandas as pd
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

42959


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [16]:
add_histone = (
    "INSERT INTO histone "
    "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
    "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
)
add_histone_description = (
    "INSERT INTO histone_description "
    "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
    "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
)
# add_publication = (
#     "INSERT INTO publication "
#     "(id, title, doi, author, year) "
#     "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
# )
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
# add_alternate_names = (
#     "INSERT INTO alternative_name "
#     "(name, taxonomy, gene, splice, histone) "
#     "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
# )
# add_histone_has_publication = (
#     "INSERT INTO histone_has_publication "
#     "(histone_id, publication_id) "
#     "VALUES (%s, %s)"
# )

# Change taxonomic span for Capstones

In [6]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Capstone"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
3,Capstone,variant_group,"Archaea, Bacteria, Viruses","2157, 2, 10239",211.0,Singlet


In [7]:
query = f"UPDATE histone SET taxonomic_span='Archaea', taxonomic_span_id=2157 WHERE id='Capstone'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Capstone"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
3,Capstone,variant_group,Archaea,2157,211.0,Singlet


In [8]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Capstone'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Capstone,variant_group,Archaea,2157,211,Singlet,211,Capstones are histone variants found in archae...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Delete nodes Archaeal, Viral

In [9]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Archaeal"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
0,Archaeal,type,null,null,1.0,None


In [10]:
query = f"DELETE FROM histone WHERE id='Archaeal'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Archaeal"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent


In [11]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Viral"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
210,Viral,type,null,null,7.0,None


In [12]:
query = f"DELETE FROM histone WHERE id='Viral'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Viral"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent


# Add Coiled-coil

In [17]:
data_histone = {
    "id": "Coiled-coil",
    "level": 'variant_group',
    "taxonomic_span": 'Archaea',
    "taxonomic_span_id": '2157',
    "description": None,
    "parent": 'Bridge',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [18]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Coiled-coil"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
76,Coiled-coil,variant_group,Archaea,2157,NaN,Bridge


In [23]:
query = f"UPDATE histone SET level='variant' WHERE id='Coiled-coil'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Coiled-coil"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
76,Coiled-coil,variant,Archaea,2157,NaN,Bridge


In [24]:
histone_description_summary = 'Coiled-coil (CC) histones are a type of histones that have been identified bioinformatically in archaea [schwab_histones_2024]. These histones contain a long α-helix at their C-terminus, which forms a coiled-coil bundle within a tetrameric structure. Predicted by AlphaFold2, they bind two DNA strands through dimers, demonstrating bridging activity. Experimental observations have revealed that the DNA bridging activity of CC histones increases in a concentration-dependent manner, with higher concentrations of CC histones (referred to as HMfC) from M. fervidus resulting in greater activity [schwab_histones_2024]. CC histones are distinguished by the presence of conserved positively charged residues (R32, K35) on the α1-helix, which facilitate interaction with DNA [schwab_histones_2024].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Coiled-coil'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [25]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Coiled-coil'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Coiled-coil,variant,Archaea,2157,217,Bridge,217,Coiled-coil (CC) histones are a type of histon...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add Bridge_(Methanococcales)

In [26]:
data_histone = {
    "id": "Bridge_(Methanococcales)",
    "level": 'variant_group',
    "taxonomic_span": 'Methanococcales',
    "taxonomic_span_id": '2182',
    "description": None,
    "parent": 'Bridge',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [27]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Bridge_(Methanococcales)"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
2,Bridge_(Methanococcales),variant_group,Methanococcales,2182,NaN,Bridge


In [28]:
query = f"UPDATE histone SET level='variant' WHERE id='Bridge_(Methanococcales)'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Bridge_(Methanococcales)"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
2,Bridge_(Methanococcales),variant,Methanococcales,2182,NaN,Bridge


In [29]:
histone_description_summary = 'Bridge_(Methanococcales) are specialized DNA-bridging histones exclusive to archaea of the order Methanococcales. These histones were predicted and confirmed by AlphaFold2 and crystallography (e.g., MJ1647 from Methanocaldococcus jannaschii, PDB: 8BDK) [schwab_histones_2024, Ofer_dna-bridging_2023]. These histones possess a distinctive C-terminal domain that facilitates tetramerization and the formation of bridges between DNA strands. The conservation of residues such as R45 and K49 on the α1-helix, which bind to DNA, along with the absence of tails, suggests adaptation to hyperthermophilic conditions [schwab_histones_2024].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Bridge_(Methanococcales)'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [30]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Bridge_(Methanococcales)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Bridge_(Methanococcales),variant,Methanococcales,2182,218,Bridge,218,Bridge_(Methanococcales) are specialized DNA-b...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Update description for Bridge node

In [35]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Bridge'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])[['id', 'summary']].values

array([['Bridge', 214,
        'DNA-bridge histones binds two strands of DNA together to form bridges.']],
      dtype=object)

In [36]:
query = f"UPDATE histone_description SET summary='DNA-bridge histones binds two strands of DNA together to form bridges. In contrast to classical histones, these variants do not wrap around DNA, but rather stabilize long-range interactions, thereby contributing to the three-dimensional organization of the genome [schwab_histones_2024].' WHERE id=214"
cursor.execute(query)

query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Bridge'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])['summary'].values

array(['DNA-bridge histones binds two strands of DNA together to form bridges. In contrast to classical histones, these variants do not wrap around DNA, but rather stabilize long-range interactions, thereby contributing to the three-dimensional organization of the genome [schwab_histones_2024].'],
      dtype=object)

In [37]:
# Make sure data is committed to the database
conn.commit()

# Update taxonomy span for Nucleosomal histones

In [38]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Nucleosomal"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
200,Nucleosomal,variant_group,"Archaea, Bacteria","2157, 2",213.0,Singlet


In [39]:
query = f"UPDATE histone SET taxonomic_span='Archaea', taxonomic_span_id=2157 WHERE id='Nucleosomal'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Nucleosomal"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
200,Nucleosomal,variant_group,Archaea,2157,213.0,Singlet


In [37]:
# Make sure data is committed to the database
conn.commit()

# Add RdgC-histones

In [40]:
data_histone = {
    "id": "RdgC-histones",
    "level": 'variant',
    "taxonomic_span": 'Bacteria',
    "taxonomic_span_id": '2',
    "description": None,
    "parent": 'Bridge',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [42]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "RdgC-histones"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
203,RdgC-histones,variant,Bacteria,2,NaN,Bridge


In [43]:
histone_description_summary = 'RdgC histones have been identified in certain Bacillus and Halobacteria species through bioinformatic methods including AlphaFold2 structure prediction, sequence clustering (CLANS) and hidden Markov models (HMM) [schwab_histones_2024]. These histones are functionally related to DNA recombination and interaction with proteins (e.g., RecA). RdgC histones are encoded in operons together with RdgC-like proteins and transmembrane domains, suggesting a role in specialized processes other than classical DNA compacting. Despite the low sequence identity, the key DNA-binding residues (e.g., K34) on the α1-helix are retained, thereby confirming their classification within the histone superfamily [schwab_histones_2024].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='RdgC-histones'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [44]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='RdgC-histones'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,RdgC-histones,variant,Bacteria,2,219,Bridge,219,RdgC histones have been identified in certain ...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add Phage-histones

In [46]:
data_histone = {
    "id": "Phage-histones",
    "level": 'variant',
    "taxonomic_span": 'Bacteria (?), Viruses',
    "taxonomic_span_id": '2 (?), 10239',
    "description": None,
    "parent": 'Bridge',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [47]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Phage-histones"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
202,Phage-histones,variant,"Bacteria (?), Viruses","2 (?), 10239",NaN,Bridge


In [48]:
histone_description_summary = 'Phage histones are histones identified in DNA-containing bacteriophages (e.g., order Caudovirales) and in some bacterial metagenomes (might be a prophage) using using bioinformatic methods including AlphaFold2 structure prediction, sequence clustering (CLANS) and hidden Markov models (HMM) [schwab_histones_2024]. They contain a histone fold domain at the N-terminus and an α-helical domain at the C-terminus for tetramerization. Phage histones are involved in viral DNA packaging but do not form nucleosomes like eukaryotic histones, it might bridge DNA through C-terminal domain [schwab_histones_2024]. Notably, these histones exhibit low sequence conservation, yet they retain critical DNA-binding residues, such as K10 and K49, which are pivotal for their function [schwab_histones_2024].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Phage-histones'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [49]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Phage-histones'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Phage-histones,variant,"Bacteria (?), Viruses","2 (?), 10239",220,Bridge,220,Phage histones are histones identified in DNA-...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add Capstone_(Methanosphaera)

In [50]:
data_histone = {
    "id": "Capstone_(Methanosphaera)",
    "level": 'variant',
    "taxonomic_span": 'Archaea',
    "taxonomic_span_id": '2157',
    "description": None,
    "parent": 'Bridge',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [51]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Capstone_(Methanosphaera)"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
4,Capstone_(Methanosphaera),variant,Archaea,2157,NaN,Bridge


In [52]:
query = f"UPDATE histone SET parent='Capstone' WHERE id='Capstone_(Methanosphaera)'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Capstone_(Methanosphaera)"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
4,Capstone_(Methanosphaera),variant,Archaea,2157,NaN,Capstone


In [53]:
query = f"UPDATE histone SET taxonomic_span='Methanosphaera', taxonomic_span_id=2316  WHERE id='Capstone_(Methanosphaera)'"
cursor.execute(query)

query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Capstone_(Methanosphaera)"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
4,Capstone_(Methanosphaera),variant,Methanosphaera,2316,NaN,Capstone


In [54]:
histone_description_summary = 'Capostones found in Methanosphaera that contain a hydrophobic amino acid in a specific position have been identified as crucial for tetramer formation [marc_archaeal_2002].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Capstone_(Methanosphaera)'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [55]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Capstone_(Methanosphaera)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Capstone_(Methanosphaera),variant,Methanosphaera,2316,221,Capstone,221,Capostones found in Methanosphaera that contai...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Add Capstones_(Methanobrevibacter)

In [56]:
data_histone = {
    "id": "Capstones_(Methanobrevibacter)",
    "level": 'variant',
    "taxonomic_span": 'Methanobrevibacter',
    "taxonomic_span_id": '2172 ',
    "description": None,
    "parent": 'Capstone',
}
cursor.execute(add_histone, data_histone)

# Make sure data is committed to the database
conn.commit()

In [58]:
query = "SELECT * FROM histone"
cursor.execute(query)
histone_df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
histone_df[histone_df["id"] == "Capstones_(Methanobrevibacter)"]

,id,level,taxonomic_span,taxonomic_span_id,description,parent
4,Capstones_(Methanobrevibacter),variant,Methanobrevibacter,2172,NaN,Capstone


In [59]:
histone_description_summary = 'Capostones found in Methanobrevibacter and contain a negatively amino acid in a specific position have been identified as crucial for tetramer formation [marc_archaeal_2002].'
data_histone_description = (histone_description_summary,) + (None,) * 13
cursor.execute(add_histone_description, data_histone_description)

histone_description_id = cursor.lastrowid
query = f"UPDATE histone SET description={histone_description_id} WHERE id='Capstones_(Methanobrevibacter)'"
cursor.execute(query)

# Make sure data is committed to the database
conn.commit()

In [60]:
query = (
    "SELECT * FROM histone h LEFT JOIN histone_description hd "
    "ON h.description = hd.id "
    "WHERE h.id='Capstones_(Methanobrevibacter)'"
)
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
0,Capstones_(Methanobrevibacter),variant,Methanobrevibacter,2172,222,Capstone,222,Capostones found in Methanobrevibacter and con...,None,None,...,None,None,None,None,None,None,None,None,None,None


# Close connections

In [61]:
cursor.close()
conn.close()
tunnel.stop()